# Simple Segmentation Model


In [4]:
import os
import pandas as pd
import numpy as np 
def load_chants(test_chants_file = "test-chants.csv", 
                train_chants_file = "train-chants.csv",
                test_repr_pitch_file = "test-representation-pitch.csv",
                train_repr_pitch_file = "train-representation-pitch.csv"):
    test_chants = pd.read_csv(test_chants_file, index_col='id')
    train_chants = pd.read_csv(train_chants_file, index_col='id')
    chants = pd.concat([train_chants, test_chants])
    pitch_repr_test = pd.read_csv(test_repr_pitch_file, index_col='id')
    pitch_repr_train = pd.read_csv(train_repr_pitch_file, index_col='id')
    pitch_representations = pd.concat([pitch_repr_train, pitch_repr_test])

    return chants, pitch_representations

def prepare_dataset():
    chants, pitch_repr = load_chants()
    X, y = [], []
    for segments, mode, id_pitches, id_chant in zip(pitch_repr["1-mer"], 
                                                chants['mode'], 
                                                pitch_repr.index, 
                                                chants.index):
        if not id_pitches == id_chant:
            raise ValueError("IDs of features and modes are not equal!")
        X.append(segments.replace(' ', ''))
        y.append(str(mode))

    return np.array(X), np.array(y)

In [6]:
import random
class SimpleSegmentationModel():
  def __init__(self):
    # dictionary of melody string and its counts over all documents (as integer)
    self.melody_counts = {} 
    # number of all segments, the sum over all counts 
    self.total_segments = 0 
    # dictionaryof melody strings and its hashset of chants that contains
    # this melody
    self.melody_in_chants = {}
    # total number of chants
    self.chant_count = 0


  def predict_segments(self, chants, iterations = 100, 
                       epsilon = 0.05, mu = 5, sigma = 2, 
                       alpha=0.0001, print_each = 5):
    # Do init segmentation, generate model's dictionaries (melody_counts, ...)
    init_segmentation = self.__gaus_rand_segments(chants, mu, sigma)
    # Update chant_count
    self.chant_count = len(chants)
    chant_segmentation = init_segmentation
    for i in range(iterations):
      chant_segmentation = self.__train_iteration(chant_segmentation, epsilon, alpha)
      if i%print_each == 0:
        print("{}. Iteration".format(i))
        top25_melodies = sorted(self.melody_counts, key=self.melody_counts.get, reverse=True)[:30]
        print("\t\t\t", top25_melodies)
        #for topmel in top25_melodies:
        #  print("\t\t\t{}".format(topmel))
    return chant_segmentation
      

  def __gaus_rand_segments(self, chants, mu, sigma):
    rand_segments = []
    for chant_id, chant in enumerate(chants):
      new_chant_segments = []
      i = 0
      while i != len(chant):
        # Find new segment
        new_len = max(int(random.gauss(mu, sigma)), 1)
        k = min(i+new_len, len(chant))
        new_chant_segments.append(chant[i:k])
        last_added_segment = new_chant_segments[-1]
        # Update melody_counts
        if last_added_segment in self.melody_counts:
          self.melody_counts[last_added_segment] += 1
        else:
          self.melody_counts[last_added_segment] = 1
        # Update total_segments count
        self.total_segments += 1
        # Update melody_in_chants
        if last_added_segment in self.melody_in_chants:
          self.melody_in_chants[last_added_segment].add(chant_id)
        else:
          self.melody_in_chants[last_added_segment] = {chant_id}
        # Update i index
        i = k
      rand_segments.append(new_chant_segments)
    return rand_segments

  def __train_iteration(self, segmented_chants, epsilon, alpha):
    new_segmented_chants = []
    join_prev_melody = None
    for chant_id, segments in enumerate(segmented_chants):
      # reset melody_in_chants
      for melody in segments:
        if chant_id in self.melody_in_chants[melody]:
          self.melody_in_chants[melody].remove(chant_id)


      new_segments = []
      for melody in segments:
        self.total_segments -= 1
        self.melody_counts[melody] -= 1

        if join_prev_melody == None:
          # How many documents contains this melody
          chant_frequency = len(self.melody_in_chants[melody])/self.chant_count

          if chant_frequency > epsilon or len(melody) <= 1: 
            # Do nothing, pass the melody to the next stage for joining
            join_prev_melody = melody
          else:
            # Find the best splitting
            max_prob = 0
            left = ""
            right = ""
            for split_point in range(1, len(melody)):
              new_left = melody[:split_point]
              new_right = melody[split_point:]
              left_freq = alpha
              right_freq = alpha
              if new_left in self.melody_counts:
                left_freq += (self.melody_counts[new_left]/self.total_segments)
              if new_right in self.melody_counts:
                right_freq += (self.melody_counts[new_right]/self.total_segments)
              
              if max_prob < left_freq * right_freq:
                max_prob = left_freq * right_freq
                left = new_left
                right = new_right
            # Joining melody with the previous one
            new_segments.append(left)
            new_segments.append(right)
            # Update total_segments count
            self.total_segments += 2
            # Update melody_counts
            if left in self.melody_counts:
              self.melody_counts[left] += 1
            else:
              self.melody_counts[left] = 1
            if right in self.melody_counts:
              self.melody_counts[right] += 1
            else:
              self.melody_counts[right] = 1
        else:
          # Joining melody with the previous one
          new_segments.append(join_prev_melody + melody)
          # Update total_segments count
          self.total_segments += 1
          # Update melody_counts
          if join_prev_melody + melody in self.melody_counts:
            self.melody_counts[join_prev_melody + melody] += 1
          else:
            self.melody_counts[join_prev_melody + melody] = 1
          join_prev_melody = None
          
      # In case of we were about to join melody which is last
      if join_prev_melody != None:
        new_segments.append(join_prev_melody)
        # Update total_segments count
        self.total_segments += 1
        # Update melody_counts
        if join_prev_melody in self.melody_counts:
          self.melody_counts[join_prev_melody] += 1
        else:
          self.melody_counts[join_prev_melody] = 1
        join_prev_melody = None

      # Update melody_in_chants
      for melody in new_segments:
        if melody in self.melody_in_chants:
          self.melody_in_chants[melody].add(chant_id)
        else:
          self.melody_in_chants[melody] = {chant_id}

      new_segmented_chants.append(new_segments)
    return new_segmented_chants

In [7]:
# Get Data
X, y = prepare_dataset()
# Init model
model = SimpleSegmentationModel()
# Train and Fit model
final_segmentation = model.predict_segments(X)

0. Iteration
			 ['g', 'h', 'f', 'k', 'd', 'l', 'e', 'hg', 'j', 'gh', 'fe', 'kj', 'fgh', 'cd', 'gf', 'fed', 'c', 'gg', 'hgf', 'kk', 'ghg', 'dd', 'ff', 'm', 'ed', 'kjh', 'ghgf', 'ef', 'jh', 'hh']
5. Iteration
			 ['g', 'h', 'f', 'd', 'k', 'l', 'e', 'gg', 'gf', 'ef', 'hg', 'kj', 'cd', 'fed', 'gh', 'dd', 'm', 'fgh', 'j', 'c', 'kjh', 'hk', 'kk', 'jk', 'lk', 'dcd', 'kkj', 'hgfg', 'khg', 'dc']
10. Iteration
			 ['g', 'h', 'f', 'd', 'k', 'l', 'e', 'gg', 'hg', 'ef', 'fgh', 'j', 'cd', 'fed', 'kj', 'm', 'dd', 'hgh', 'kjh', 'c', 'gf', 'kk', 'khg', 'hgg', 'dcd', 'jk', 'ghgf', 'dc', 'kkj', 'hgfg']
15. Iteration
			 ['g', 'h', 'f', 'd', 'k', 'l', 'e', 'hg', 'gg', 'ef', 'm', 'cd', 'dd', 'fed', 'fgh', 'kj', 'j', 'kjh', 'gf', 'kk', 'c', 'dcd', 'hgh', 'khg', 'hk', 'ghgf', 'jk', 'dc', 'kkj', 'lk']
20. Iteration
			 ['g', 'h', 'f', 'd', 'k', 'l', 'e', 'hg', 'gg', 'fgh', 'cd', 'ef', 'j', 'fed', 'kj', 'm', 'dd', 'hgh', 'kjh', 'kk', 'hgg', 'c', 'gf', 'khg', 'dcd', 'lm', 'ghgf', 'jk', 'dc', 'kkj']
25. Iterati

In [8]:
print(final_segmentation)

[['h', 'ghghgggj', 'k', 'll', 'k', 'kkkj', 'khgggjghj', 'k', 'll', 'hjk', 'h', 'j', 'ggg', 'h', 'kkkk', 'kkjk', 'hg'], ['ggk', 'khgh', 'g', 'fgh', 'g', 'gghgf', 'gghff', 'g', 'fhjkl', 'kjk', 'lkhkj', 'hgghg', 'fghh', 'gg'], ['fggfgghf', 'f', 'gfff', 'fi', 'i', 'hggiki', 'hghgffggf', 'f', 'ii', 'hghg', 'g', 'ghg', 'f', 'gfedC', 'f', 'fgfghfgff', 'hh', 'fgh', 'gf'], ['d', 'ef', 'ggefeddfe', 'fgdfe', 'f', 'dcd', 'f', 'fEc', 'fd', 'df', 'fffe', 'cdd'], ['de', 'fggefE', 'ddfefggd', 'fef', 'd', 'cc', 'dfec', 'e', 'dfffe', 'cd'], ['d', 'efggefE', 'fd', 'e', 'fgH', 'gdfefd', 'dccd', 'f', 'E', 'c', 'c', 'ddfffe', 'cd'], ['defggffe', 'Dd', 'dfef', 'gg', 'd', 'f', 'ef', 'dcdf', 'fEcfddcdf', 'f'], ['d', 'ef', 'ggf', 'fedd', 'f', 'e', 'fggdfe', 'fdc', 'd', 'fefc', 'fdd', 'f', 'ffe', 'cdd'], ['def', 'ggefEd', 'dc', 'fgh', 'gfg', 'fgfedc', 'eg', 'efdcd', 'd', 'fghh', 'hhgfghgf'], ['d', 'ef', 'ggefedd', 'f', 'ef', 'ggdf', 'efdc', 'd', 'fecfdd', 'fffecd'], ['ddc', 'f', 'gf', 'hhi', 'h', 'ggfghgge', 'g'

In [9]:
final_melodies = {}
for melody in model.melody_counts:
  if model.melody_counts[melody] > 0:
    final_melodies[melody] = model.melody_counts[melody]

print(dict(sorted(final_melodies.items(), key=lambda item: item[1], reverse=True)))

{'g': 19896, 'h': 16461, 'f': 14842, 'd': 9444, 'k': 8365, 'l': 6866, 'e': 6026, 'hg': 3032, 'gg': 2442, 'ef': 1998, 'cd': 1762, 'm': 1717, 'dd': 1545, 'kj': 1507, 'fed': 1458, 'fgh': 1378, 'j': 1377, 'kk': 1305, 'kjh': 1300, 'gf': 1251, 'c': 1209, 'dcd': 1097, 'khg': 1081, 'hgh': 1067, 'hk': 1039, 'ghgf': 1001, 'jk': 940, 'kkj': 921, 'lk': 917, 'dc': 909, 'hgg': 866, 'hh': 852, 'jh': 766, 'll': 751, 'gfe': 747, 'lm': 743, 'ee': 728, 'ff': 722, 'i': 716, 'hgf': 702, 'fghh': 695, 'df': 694, 'hhh': 692, 'ggg': 684, 'kkk': 679, 'fgg': 671, 'ml': 667, 'fg': 664, 'def': 654, 'fgf': 646, 'hgfg': 645, 'gh': 641, 'ed': 639, 'fghg': 618, 'lkj': 612, 'jkl': 609, 'kl': 607, 'kjk': 599, 'kh': 599, 'gfed': 598, 'n': 594, 'klk': 590, 'fedd': 589, 'hhg': 569, 'ghg': 565, 'hj': 528, 'efg': 521, 'de': 519, 'ffe': 514, 'ddc': 512, 'hgfgh': 487, 'G': 485, 'fe': 483, 'ddd': 472, 'hG': 472, 'cdd': 470, 'E': 465, 'cdf': 463, 'hghg': 449, 'gfgh': 448, 'fgfe': 446, 'fff': 446, 'ggk': 445, 'hggg': 445, 'jhg': 